In [54]:
import sqlite3
import pandas as pd
pd.set_option('display.max_colwidth', None)  # para ver el texto completo


In [25]:
connection = sqlite3.connect("./data/sql-murder-mystery.db")
cursor_db = connection.cursor()
def from_query_to_df(query):
    cursor_db.execute(query)
    resultado = cursor_db.fetchall()
    columnas = [description[0] for description in cursor_db.description]

    df = pd.DataFrame(resultado,columns=columnas)

    return df

In [122]:
print("Lo que recordamos es que el crimen se cometió en SQL City, el 15 de Enero de 2018 y se trató de un asesinato.\n" \
"Con esta información vamos a investigar en el archivo de crímenes de la policía...\n")
city = "SQL City"
fecha = 20180115
tipo = "murder"
table_crimes = "crime_scene_report"
query = f'''SELECT * FROM {table_crimes} 
where date = {fecha} AND city LIKE "{city}" AND type LIKE "{tipo}"
'''
evento_df = from_query_to_df(query)
evento_df

Lo que recordamos es que el crimen se cometió en SQL City, el 15 de Enero de 2018 y se trató de un asesinato.
Con esta información vamos a investigar en el archivo de crímenes de la policía...



,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [131]:
'''Security footage shows that there were 2 witnesses.
 The first witness lives at the last house on "Northwestern Dr".
   The second witness, named Annabel, lives somewhere on "Franklin Ave".
'''
print("Los registros nos dicen que se encontraron dos testigos en la escena del crimen y donde podemos localizar a estos testigos.\n" \
"Vamos a revisar el registro ciudadano de SQLCity para localizarles.\n")
street_witness_1 = "Northwestern Dr"
name_witness_2 = "Annabel"
street_witness_2 = "Franklin Ave"
table_citizens = 'person'
query = f'''
select * from {table_citizens}
WHERE address_street_name LIKE "%{street_witness_1}%"
AND address_number = (SELECT MAX(address_number) FROM {table_citizens} WHERE address_street_name LIKE "%{street_witness_1}%")
order by address_number DESC
'''
witness_1 = from_query_to_df(query)
# witness_1 = witness_1.iloc[0]
print("El  primer testigo tiene los siguientes datos: \n")
display(witness_1)

query = f'''
select * from {table_citizens}
WHERE address_street_name LIKE "%{street_witness_2}%"
AND name LIKE "%{name_witness_2}%"
'''
witness_2 = from_query_to_df(query)
print("Nuestro segundo testigo está localizado, estos son sus datos: \n")
display(witness_2)


Los registros nos dicen que se encontraron dos testigos en la escena del crimen y donde podemos localizar a estos testigos.
Vamos a revisar el registro ciudadano de SQLCity para localizarles.

El  primer testigo tiene los siguientes datos: 



,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


Nuestro segundo testigo está localizado, estos son sus datos: 



,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [140]:
print("¡Bien!, ya tenemos la información personal de los testigos y su identificación.\n" \
"Podemos ir al registro de interrogatorios y extraer sus declaraciones.\n")

id_witness_1 = witness_1.iloc[0].id
id_witness_2 = witness_2.iloc[0].id
table_interview = "interview"

query = f'''SELECT * FROM {table_interview} WHERE person_id = {id_witness_1}
'''
interview_1 = from_query_to_df(query)
print(f"Esta es la declaración del primer testigo {witness_1.iloc[0]["name"]}\n{interview_1.iloc[0].transcript}\n")

query = f'''SELECT * FROM {table_interview} WHERE person_id = {id_witness_2}
'''
interview_2 = from_query_to_df(query)
print(f"Esta es la declaración de nuestra segunda testigo {witness_2.iloc[0]["name"]}\n{interview_2.iloc[0].transcript}\n")


¡Bien!, ya tenemos la información personal de los testigos y su identificación.
Podemos ir al registro de interrogatorios y extraer sus declaraciones.

Esta es la declaración del primer testigo Morty Schapiro
I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".

Esta es la declaración de nuestra segunda testigo Annabel Miller
I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.



In [142]:
# Ahora con la información de los testigos consultaremos en la tabla de usuarios del gimnasio
print(f'''Basándonos en la declaración de {witness_1.iloc[0]["name"]}, que nos parece más detallada y precisa.\n
Vamos a pedir los registros de los miembros del gimnasio.\n''')
membresia = "gold"
id_socio = "48Z%"
matricula = "%H42W%"
fecha_acceso = ''
table_gym_members = "get_fit_now_member"
query = f'''SELECT * FROM {table_gym_members} 
WHERE membership_status = "{membresia}" AND id LIKE "{id_socio}"
'''
sospechosos_df = from_query_to_df(query)
print("Tenemos dos sospechosos que encajan con lo que nos dijo el primer testigo:\n")
sospechosos_df

Basándonos en la declaración de Morty Schapiro, que nos parece más detallada y precisa.

Vamos a pedir los registros de los miembros del gimnasio.

Tenemos dos sospechosos que encajan con lo que nos dijo el primer testigo:



,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [145]:
print("Con los datos del gimnasio y la información aportada sobre el vehículo, podemos cotejar la matricula\n" \
"con el registro de conductores y localizar al propietario del vehículo, pudiendo descartar a un sospechoso.")
table_drivers = "drivers_license"
query = f'''SELECT p.*
FROM {table_citizens} as p
JOIN {table_drivers} as d ON p.license_id = d.id
WHERE d.plate_number LIKE "{matricula}"
AND p.id IN (SELECT person_id FROM {table_gym_members} 
WHERE membership_status = "{membresia}" AND id LIKE "{id_socio}")
'''
sospechoso_final = from_query_to_df(query)
print("Nuestro sospechoso final es:\n")
display(sospechoso_final)

Con los datos del gimnasio y la información aportada sobre el vehículo, podemos cotejar la matricula
con el registro de conductores y localizar al propietario del vehículo, pudiendo descartar a un sospechoso.
Nuestro sospechoso final es:



,id,name,license_id,address_number,address_street_name,ssn
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [146]:
nombre_sospechoso = str(sospechoso_final["name"].loc[0])
print(f"Vamos a detener a {nombre_sospechoso} y comprobar su culpabilidad haciéndole declarar.")
tabla_solucion = "solution"
query_insert = f'''INSERT INTO {tabla_solucion} VALUES (1, "{nombre_sospechoso}")'''
cursor_db.execute(query_insert)
query = f'''SELECT value FROM {tabla_solucion}
'''
solucion = from_query_to_df(query)
print(solucion)

Vamos a detener a Jeremy Bowers y comprobar su culpabilidad haciéndole declarar.
                                                                                                                                                                                                                                                                                                                                                                                                value
0  Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.


In [149]:
# Solución del verdadero culpable en dos querys
# Primera query para ver la declaración y extraer info
print(f"{nombre_sospechoso} ha confesado el crimen y nos ha dado información sobre quién le contrato.\nVamos a ver su declaración.")
id_culpable = sospechoso_final.id.iloc[0]

query = f'''SELECT * FROM {table_interview} WHERE person_id = {id_culpable}
'''
declaración_asesino = from_query_to_df(query)
display(declaración_asesino)

Jeremy Bowers ha confesado el crimen y nos ha dado información sobre quién le contrato.
Vamos a ver su declaración.


,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [150]:
print(f"{nombre_sospechoso} ha cantado y nos ha dado información muy valiosa para localizar a quién le contrató.")
hair_color = "red"
car_brand = "Tesla"
car_model = "Model S"
event_name = "SQL Symphony Concert"
event_date = 201712
table_fb = 'facebook_event_checkin'
query =f'''SELECT p.*,fb.*,d.* 
FROM {table_citizens} as p
JOIN {table_drivers} as d ON p.license_id = d.id
JOIN {table_fb} as fb ON p.id = fb.person_id
WHERE fb.event_name LIKE "{event_name}"
AND fb.date LIKE "{event_date}%"
AND d.car_make LIKE "{car_brand}"
AND d.car_model LIKE "{car_model}"
AND d.hair_color = "{hair_color}"
GROUP BY p.id
'''
culpable_definitivo = from_query_to_df(query)
print(f"Con toda la información aportada por {nombre_sospechoso} hemos podido localizar a {culpable_definitivo.iloc[0]["name"]} como la persona que encargó el asesinato.")
display(culpable_definitivo)

Jeremy Bowers ha cantado y nos ha dado información muy valiosa para localizar a quién le contrató.
Con toda la información aportada por Jeremy Bowers hemos podido localizar a Miranda Priestly como la persona que encargó el asesinato.


,id,name,license_id,address_number,address_street_name,ssn,person_id,event_id,event_name,date,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,99716,1143,SQL Symphony Concert,20171206,202298,68,66,green,red,female,500123,Tesla,Model S


In [152]:
# Con los datos del sospechoso final, vamos a comprobar si es el culpable o no
nombre_culpable_final = str(culpable_definitivo["name"].loc[0])
query_insert = f'''INSERT INTO {tabla_solucion} VALUES (1, "{nombre_culpable_final}")'''
cursor_db.execute(query_insert)
query = f'''SELECT value FROM {tabla_solucion}
'''
solucion = from_query_to_df(query)
print(f"Hemos detenido a {culpable_definitivo["name"].loc[0]} como la persona que orquestó todo.\n{city} es ahora un lugar más seguro.")
print(solucion)

Hemos detenido a Miranda Priestly como la persona que orquestó todo.
SQL City es ahora un lugar más seguro.
                                                                                                                                                          value
0  Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!
